In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import nibabel as nib
from tensorflow import keras
import tensorflow_addons as tfa
import random
from scipy import ndimage
from tensorflow import keras
import os
import pydicom as dcm
import scipy
import cv2
from tqdm import tqdm

In [2]:
def window_image(img, minn,maxx, intercept, slope, rescale=True):
    img = (img*slope +intercept) 
    
    img[img<minn] = minn 
    img[img>maxx] = maxx 
    if rescale: 
        img = (img - minn) / (maxx - minn)
    return img
    
def get_first_of_dicom_field_as_int(x):
    if type(x) == dcm.multival.MultiValue: return int(x[0])
    else: return int(x)
    
def get_windowing(data):
    dicom_fields = [data[('0028','1050')].value,
                    data[('0028','1051')].value,
                    data[('0028','1052')].value,
                    data[('0028','1053')].value]
    return [get_first_of_dicom_field_as_int(x) for x in dicom_fields]

In [3]:
def channeling(img1, img2, img3):
    return np.stack([img1, img2, img3], axis=-1)

In [4]:
def load_image(path):
    data_path = '../../Downloads/rsna-2022-cervical-spine-fracture-detection/train_images/'
    UID = path.split('/')[-1][:-7]+"/"
    data = dcm.dcmread(data_path+UID+os.listdir(data_path+UID)[0])
    _, _, intercept, slope = get_windowing(data)
    img = nib.load(path)
    img = img.get_fdata()
    return img, intercept, slope

In [5]:
def load_meta(UID):
    data_path = '../../Downloads/rsna-2022-cervical-spine-fracture-detection/train_images/'
    data = dcm.dcmread(data_path+UID+'/'+os.listdir(data_path+UID)[0])
    _, _, intercept, slope = get_windowing(data)
    return intercept, slope

In [6]:
df = pd.read_csv("../../Downloads/rsna-2022-cervical-spine-fracture-detection/train.csv")

In [7]:
base_model = tf.keras.models.load_model('./feature_extractor/feature_extractor/') # (None, 256, 256, 3)

In [7]:
n_image_list = np.arange(0, df.shape[0])
random_n_image_list = np.random.choice(n_image_list, df.shape[0], replace=False)

In [9]:
!mkdir extracted_features

A subdirectory or file extracted_features already exists.


In [10]:
for image in tqdm(df['StudyInstanceUID']):
        try:
                co = df[df['StudyInstanceUID']==image]['patient_overall'].values[0]
                c1 = df[df['StudyInstanceUID']==image]['C1'].values[0]
                c2 = df[df['StudyInstanceUID']==image]['C2'].values[0]
                c3 = df[df['StudyInstanceUID']==image]['C3'].values[0]
                c4 = df[df['StudyInstanceUID']==image]['C4'].values[0]
                c5 = df[df['StudyInstanceUID']==image]['C5'].values[0]
                c6 = df[df['StudyInstanceUID']==image]['C6'].values[0]
                c7 = df[df['StudyInstanceUID']==image]['C7'].values[0]
                ct = nib.load('../storage/'+image+'.nii.gz')
                ct = ct.get_fdata()
                ct = scipy.ndimage.zoom(ct, (256/ct.shape[0],1,1), order=1)
                ct = np.transpose(ct, axes=[2,0,1])[68:188:2]
                intercept, slope = load_meta(image)
                ct1 = window_image(ct,100,600,intercept,slope)
                ct2 = window_image(ct,600,1100,intercept,slope)
                ct3 = window_image(ct,1100,1600,intercept,slope)
                ct = channeling(ct1, ct2, ct3)
                features = np.array(base_model(ct))
                np.save('../RSNA-2022-Cervical-Spine-Fracture-Detection/extracted_features/'+image+'.npy', features)
        except:
                continue
        

100%|██████████| 2019/2019 [1:45:44<00:00,  3.14s/it]


: 

In [8]:
def data_gen():
    for image in df['StudyInstanceUID'][:1800]:
        try:
            co = df[df['StudyInstanceUID']==image]['patient_overall'].values[0]
            c1 = df[df['StudyInstanceUID']==image]['C1'].values[0]
            c2 = df[df['StudyInstanceUID']==image]['C2'].values[0]
            c3 = df[df['StudyInstanceUID']==image]['C3'].values[0]
            c4 = df[df['StudyInstanceUID']==image]['C4'].values[0]
            c5 = df[df['StudyInstanceUID']==image]['C5'].values[0]
            c6 = df[df['StudyInstanceUID']==image]['C6'].values[0]
            c7 = df[df['StudyInstanceUID']==image]['C7'].values[0]
            
            features = np.load('../RSNA-2022-Cervical-Spine-Fracture-Detection/extracted_features/'+image+'.npy')

            yield ({"input_1": features},
                  {'co': co, 
                                        'c1': c1,
                                        'c2': c2,
                                        'c3': c3,
                                        'c4': c4,
                                        'c5': c5,
                                        'c6': c6,
                                        'c7': c7,},
                      {'co': 14, 
                                    'c1': c1+1,
                                    'c2': c2+1,
                                    'c3': c3+1,
                                    'c4': c4+1,
                                    'c5': c5+1,
                                    'c6': c6+1,
                                    'c7': c7+1})
        except:
            continue


def test_gen():
    for image in df['StudyInstanceUID'][1880:]:
        try:
            co = df[df['StudyInstanceUID']==image]['patient_overall'].values[0]
            c1 = df[df['StudyInstanceUID']==image]['C1'].values[0]
            c2 = df[df['StudyInstanceUID']==image]['C2'].values[0]
            c3 = df[df['StudyInstanceUID']==image]['C3'].values[0]
            c4 = df[df['StudyInstanceUID']==image]['C4'].values[0]
            c5 = df[df['StudyInstanceUID']==image]['C5'].values[0]
            c6 = df[df['StudyInstanceUID']==image]['C6'].values[0]
            c7 = df[df['StudyInstanceUID']==image]['C7'].values[0]
            features = np.load('../RSNA-2022-Cervical-Spine-Fracture-Detection/extracted_features/'+image+'.npy')

            yield ({"input_1": features},
                  {'co': co, 
                                        'c1': c1,
                                        'c2': c2,
                                        'c3': c3,
                                        'c4': c4,
                                        'c5': c5,
                                        'c6': c6,
                                        'c7': c7,},
                      {'co': 14, 
                                    'c1': c1+1,
                                    'c2': c2+1,
                                    'c3': c3+1,
                                    'c4': c4+1,
                                    'c5': c5+1,
                                    'c6': c6+1,
                                    'c7': c7+1})
        except:
            continue

In [9]:
dataset = tf.data.Dataset.from_generator(
     data_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8},
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([60,2048])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])},
    {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

testset = tf.data.Dataset.from_generator(
     test_gen,
     ({"input_1":tf.float32}, 
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8},
     {'co':tf.int8,
     'c1':tf.int8,
     'c2':tf.int8,
     'c3':tf.int8,
     'c4':tf.int8,
     'c5':tf.int8,
     'c6':tf.int8,
     'c7':tf.int8}),
    ({"input_1":tf.TensorShape([60,2048])}, 
     {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])},
    {"co": tf.TensorShape([]), 
     "c1": tf.TensorShape([]),
     "c2": tf.TensorShape([]),
     "c3": tf.TensorShape([]),
     "c4": tf.TensorShape([]),
     "c5": tf.TensorShape([]),
     "c6": tf.TensorShape([]),
     "c7": tf.TensorShape([])})
)

In [10]:
dataset = dataset.batch(5)
dataset = dataset.prefetch(1)


testset = testset.batch(5)
testset = testset.prefetch(1)

In [11]:
input1 = keras.Input((60, 2048),name = "input_1")

x = keras.layers.BatchNormalization()(input1)
x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=True))(x)
x = keras.layers.Dense(2048, activation='relu')(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(2048, return_sequences=False))(x)

x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)

x0 = tf.keras.layers.Dense(256, activation='relu')(x)
x0 = tf.keras.layers.Dense(128, activation='relu')(x0)
x0 = tf.keras.layers.Dense(32, activation='relu')(x0)

x1 = tf.keras.layers.Dense(256, activation='relu')(x)
x1 = tf.keras.layers.Dense(128, activation='relu')(x1)
x1 = tf.keras.layers.Dense(32, activation='relu')(x1)

x2 = tf.keras.layers.Dense(256, activation='relu')(x)
x2 = tf.keras.layers.Dense(128, activation='relu')(x2)
x2 = tf.keras.layers.Dense(32, activation='relu')(x2)

x3 = tf.keras.layers.Dense(256, activation='relu')(x)
x3 = tf.keras.layers.Dense(128, activation='relu')(x3)
x3 = tf.keras.layers.Dense(32, activation='relu')(x3)

x4 = tf.keras.layers.Dense(256, activation='relu')(x)
x4 = tf.keras.layers.Dense(128, activation='relu')(x4)
x4 = tf.keras.layers.Dense(32, activation='relu')(x4)

x5 = tf.keras.layers.Dense(256, activation='relu')(x)
x5 = tf.keras.layers.Dense(128, activation='relu')(x5)
x5 = tf.keras.layers.Dense(32, activation='relu')(x5)

x6 = tf.keras.layers.Dense(256, activation='relu')(x)
x6 = tf.keras.layers.Dense(128, activation='relu')(x6)
x6 = tf.keras.layers.Dense(32, activation='relu')(x6)

x7 = tf.keras.layers.Dense(256, activation='relu')(x)
x7 = tf.keras.layers.Dense(128, activation='relu')(x7)
x7 = tf.keras.layers.Dense(32, activation='relu')(x7)

outputo = keras.layers.Dense(1 ,activation = 'sigmoid', name="co")(x0)
output1 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c1")(x1)
output2 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c2")(x2)
output3 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c3")(x3)
output4 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c4")(x4)
output5 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c5")(x5)
output6 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c6")(x6)
output7 = keras.layers.Dense(1 ,activation = 'sigmoid', name="c7")(x7)
                                               
s_model = keras.models.Model(inputs=input1, outputs=[outputo,output1,output2,output3,output4,output5,output6,output7])

In [12]:
def lr_schedule(epoch, lr):
    return lr/((epoch+2)/2)

In [13]:
opt = tfa.optimizers.AdamW(
    weight_decay=1e-4,
    learning_rate = 0.001,
    beta_1 = 0.9,
    beta_2 = 0.999,
    epsilon = 1e-07,
    name = 'AdamW',
)
loss = tf.keras.losses.BinaryCrossentropy(
#     label_smoothing=0.1, 
    reduction=tf.keras.losses.Reduction.AUTO,
    name='binary_crossentropy'
)
metrics = [tf.keras.metrics.AUC()]
s_model.compile(optimizer=opt,
              loss={"co": loss, 
                   "c1": loss,
                   "c2": loss,
                   "c3": loss,
                   "c4": loss,
                   "c5": loss,
                   "c6": loss,
                   "c7": loss,},
                metrics={"co": metrics})

In [14]:
!mkdir final_smodel

A subdirectory or file final_smodel already exists.


In [15]:
model_checkpoint_callback_LASSO = tf.keras.callbacks.ModelCheckpoint(
    filepath = './final_smodel/model',
    monitor="val_loss",
    save_best_only=True,
    save_weights_only=True,
    mode="auto",
)

In [ ]:
history = s_model.fit(dataset, validation_data = testset, epochs=20, callbacks=[model_checkpoint_callback_LASSO, tf.keras.callbacks.LearningRateScheduler(lr_schedule)])

In [17]:
s_model.load_weights('./final_smodel/model')

In [18]:
s_model.evaluate(testset)

28/28 [==============================] - 3s 86ms/step - loss: 12.8711 - co_loss: 9.2336 - c1_loss: 0.4555 - c2_loss: 0.6635 - c3_loss: 0.1814 - c4_loss: 0.4867 - c5_loss: 0.4134 - c6_loss: 0.6149 - c7_loss: 0.8219 - co_auc: 0.6531


[12.871099472045898,
 9.233647346496582,
 0.4555373787879944,
 0.6635385155677795,
 0.1814422905445099,
 0.48674824833869934,
 0.41338467597961426,
 0.6149333715438843,
 0.8218652009963989,
 0.6530655026435852]

In [19]:
!mkdir saved_final_smodel

In [20]:
s_model.save('./saved_final_smodel/model')

INFO:tensorflow:Assets written to: ./saved_final_smodel/model\assets


INFO:tensorflow:Assets written to: ./saved_final_smodel/model\assets
